In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# Author: Tolulope Olugboji
# Date:  October 10, 2013
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#
# Objective:   Read SAC Files [OBS DATA ONLY] from Kawakatsu Japanese project...
#              Scan a huge record database and parse event times, Conduct Quality Check,
#             and visualize seismograms semi-autonomously
#
#
# Code scans through data and displays each band pass window for the vertical seismogram
# Modules include dateTime, read_Records, signal_filtering, envelopes, output_Clear, tauP_routines
#
#

from obspy import UTCDateTime
from obspy import read
import obspy.signal
import time, sys, os
from IPython.display import display, clear_output
from obspy.taup.taup import getTravelTimes
from obspy.signal import rotate_new

def traceTagTimeSlow(threeCompStrm, travelTimeDic, pTime):
    "tag the header t0 with first arrival phase name"

    # Write Phase Name, Phase Slowness and Phase ?
    for iCmp in range(3):
        threeCompStrm[iCmp].stats.sac['t0'] = pTime
        threeCompStrm[iCmp].stats.sac['user0'] =  travelTimeDic['dT/dD']
        threeCompStrm[iCmp].stats.sac['kt0'] =  travelTimeDic['phase_name']


def traceTagQC(threeCompStrm, pTime):
    "tag the header t1, to indicate QC pass"

    # Put QC tag in here ...
    for iCmp in range(3):
        threeCompStrm[iCmp].stats.sac['t1'] = pTime

def iterateProjectData(projectDir, yearDir):
    # traverse project directory, and return directory tree for
    # data - Year, Station - Files...
    # In addition, return a list of dictionaries for Station:Orientation Codes.
    # This is required for rotating SAC Files
    #
    # Return: File List  by [Year, Station] ? What structure to use for this???
    #         List of SAC-Dictionaries with Station:Orientation_Code for each Year

    # staticOrientation[0] = 2005 ; staticOrientation[1] = 2006; staticOrientation[2] = 2007
    staticOrientation = [{'t06':228.002,
                          't08':324.55,
                          't09':13.3069,
                          't12':300.839,
                          't15':350.413,
                          't17':20.0313,
                          't18':180.928,
                          't19':299.896,
                          't21':334.879,},

                         {'t01':239.707,
                          't02':194.674,
                          't03':-0.794498,
                          't05':100.041,
                          't06':68.9703,
                          't07':325.232,
                          't08':15.5905,
                          't09':285.814,
                          't11':8.16505,
                          't12':177.762,
                          't13':223.077,},

                         {'t02':158.52,
                          't03':193.695,
                          't05':300.223,
                          't06':152.368,
                          't07':1.09913,
                          't09':295.433,
                          't12':55.9152,
                          't13':84.3507,
                          't15':241.973,
                          't16':132.348,
                          't18':353.411,}

                         ]

    projectYearDir = projectDir + yearDir

    if (yearDir == "2005/"):
        stationDirs = staticOrientation[0].keys()
        orientationCodeByYear = staticOrientation[0]
    elif (yearDir == "2006/"):
        stationDirs = staticOrientation[1].keys()
        orientationCodeByYear = staticOrientation[1]
    elif (yearDir == "2007/"):
        stationDirs = staticOrientation[2].keys()
        orientationCodeByYear = staticOrientation[2]

    print "No of stations: ", len(stationDirs), "Stations: ", stationDirs
    listStationFiles = []

    for stationDir in stationDirs:
        projectYearStationDir = projectYearDir + stationDir
        #print "Dirs: ", projectYearStationDir
        fileNames = !ls $projectYearStationDir/*Z
        #print "No of files in station, ", stationDir, " :", len(fileNames)
        listStationFiles.append(fileNames)

    return stationDirs, listStationFiles, orientationCodeByYear


    #return stations, fileLists, OrientationDict

def read3Channels(DataDir, fileMantissa, Fileformat):
    #Code snippet to take file name, and return 3component Stream.
    "Code snippet to take file name, and return 3component Stream."
    vertFile = fileMantissa+"Z";
    radFile = fileMantissa+"H1";
    transFile = fileMantissa+"H2"
    print "Names: ", vertFile, radFile, transFile;
    # read each record triplet, and display ...
    threeCompStrm = read(DataDir + '/' + vertFile);
    threeCompStrm += read(DataDir + '/' + radFile);
    threeCompStrm += read(DataDir + '/' + transFile);
    return threeCompStrm

def preProcessStream(threeCompStrm, noiseWindowSec, signalWindowSec):
    "Slice Trace, Demean, Detrend, and HighPass?"
    # noiseWindowSec : time before arrival P time
    # signalWindowSec : time after arrival P time

    #!!!! ................................. Load Depth, Distance, Then Compute Travel Time
    evntDistDeg = threeCompStrm[0].stats.sac['gcarc'];
    evntDep = threeCompStrm[0].stats.sac['evdp'];
    strtTime = threeCompStrm[0].stats['starttime'];
    tt = getTravelTimes(delta=evntDistDeg, depth = evntDep, model='ak135')
    pTime = tt[0]['time']

    biasLeft = noiseWindowSec; biasRight = signalWindowSec
    #print "Windows ...", biasLeft, biasRight, "Time ", strtTime

    #1. Slice And Tag ...
    procStream = threeCompStrm.slice(strtTime+pTime - biasLeft, strtTime+pTime+biasRight)
    traceTagTimeSlow(procStream, tt[0], biasLeft)

    # Demean, Detrend, and, HighPass? Yes with 50s [1/50 = 0.02 Hz]  *******

    lowFreq = 0.02
    procStream.detrend(type="demean"); procStream.detrend(type="simple")
    procStream.filter("highpass", freq=lowFreq)


    return procStream

def readHeaders(trace):
    "Read relevant header information ..."
    return {'gcarc': trace.stats.sac['gcarc'], 'evdp':trace.stats.sac['evdp'],
            'starttime':trace.stats['starttime'], 'o': trace.stats['sac']['o'], 'delta': trace.stats['delta'],
            'baz':trace.stats.sac['baz']}

def rotateStream2ZRT(threeCompStrm, H1Azim):
    "Load 3 component stream and rotate from ZH1H2 into ZRT."
    #rotate_new from krischer / obspy might be useful ..
    baz = threeCompStrm[1].stats.sac['baz']

    dipZcomp = -90; dipH1comp = 0.0; dipH2comp = 0.0
    azimZcomp = 0; azimH1comp = H1Azim; azimH2comp = (H1Azim + 90.0) % 360
    print "H1 azim: ", azimH1comp, "H2 azim: ", azimH2comp
    zData = threeCompStrm[0].data ; H1Data = threeCompStrm[1].data ; H2Data = threeCompStrm[2].data;
    [Z, N, E] = rotate_new.rotate2ZNE(zData, azimZcomp, dipZcomp, H1Data, azimH1comp, dipH1comp, \
                                      H2Data,azimH2comp, dipH2comp)

    [R, T] = rotate_new.rotate_NE_RT(N, E, baz)

    threeCompStrm[0].data = Z
    threeCompStrm[1].data = R
    threeCompStrm[2].data = T



def saveStream(threeCompStrm, outDir, fileMantissa):
    # Code Snippet to Take Processed Stream and Save to Hard-Disk"
    "Save Stream to Directory, Default Output is SAC file ..."
    print "Saving trace to: ", outDir+fileMantissa+".[ZRT]"

    vertFile = fileMantissa+"Z";
    radFile = fileMantissa+"R";
    transFile = fileMantissa+"T"

    if (os.path.isdir(outDir) ):
        print "Dir exists: "
        threeCompStrm[0].write(outDir + vertFile, format="SAC")
        threeCompStrm[1].write(outDir + radFile, format="SAC")
        threeCompStrm[2].write(outDir + transFile, format="SAC")
    else:
        print "Make Directory, then save.."
        os.makedirs(outDir)
        threeCompStrm[0].write(outDir + vertFile, format="SAC")
        threeCompStrm[1].write(outDir + radFile, format="SAC")
        threeCompStrm[2].write(outDir + transFile, format="SAC")


def runBandCheckQC(threeCompStrm, noiseWin, signalWin):
    from copy import deepcopy
    # run signal to noise in each window and tag phas appropriately
    #################### FREQ Windows
    cornerFreq = 0.07
    lowFreqA = 0.07
    highFreqA = 0.25
    lowFreqB = 0.25
    highFreqB = 2
    #################### End FREQ Windows

    # Make copy of stream ...
    lowFrqStrm = deepcopy(threeCompStrm)
    midFrqStrm = deepcopy(threeCompStrm)
    highFrqStrm = deepcopy(threeCompStrm)

    # Band Pass these Copies
    #lowFrqStrm.filter("lowpass", freq=cornerFreq);
    lowFrqStrm.filter("bandpass", freqmin=0.01, freqmax=0.07, corners=2);
    midFrqStrm.filter("bandpass", freqmin=lowFreqA, freqmax=highFreqA);
    highFrqStrm.filter("bandpass", freqmin=lowFreqB, freqmax=highFreqB);

    zEnvlpBands = []
    zComps = []
    allSigNoise = []

    # Pick Out vertical Components of Bands and append to list for signal analysis
    zEnvlpBands.append( obspy.signal.filter.envelope(lowFrqStrm[0].data) )
    zComps.append( lowFrqStrm[0].data )

    zEnvlpBands.append( obspy.signal.filter.envelope(midFrqStrm[0].data) )
    zComps.append( midFrqStrm[0].data )

    zEnvlpBands.append( obspy.signal.filter.envelope(highFrqStrm[0].data) )
    zComps.append( highFrqStrm[0].data )


    plt.figure(figsize=(10,4))

    # Iterate through bands and do signal to noise computation ...
    for iterBands in range( len(zEnvlpBands) ):

        plt.subplot(3,1,iterBands+1)

        zEnvlpBand = zEnvlpBands[iterBands]
        zComp = zComps[iterBands]

        t = np.linspace(0, noiseWin+signalWin, len(zEnvlpBand) )
        noise =  zEnvlpBand[t<100];  signal =  zEnvlpBand[t >100];

        sig2noise = np.average(np.absolute(signal)) / np.average(np.absolute(noise));
        allSigNoise.append( sig2noise )
        print "Signal to Noise: ", np.average(np.absolute(signal)), np.average(np.absolute(noise)), sig2noise

        plt.plot(t, zComp)
        plt.plot(t, zEnvlpBand)

        if( sig2noise > 2):
            plt.plot(t[0:len(noise)], noise, 'r')


    plt.show()
    sig2noiseMax = np.amax(allSigNoise)
    print "Maximum Signal to Noise across 3 Frequency bands: ", sig2noiseMax

    if( sig2noiseMax > 2):
        traceTagQC(threeCompStrm, noiseWin)
        tagStatus = True
    else:
        tagStatus = False

    return tagStatus;


########################### END OF FUNCTION DEFINITIONS #########################################################
#!!!!!!!!!!!!!!!!!!!!!!!!!!! BEGIN WORKFLOW HERE ...... !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
##++--- ---+ !!!!!!! Work on Work flow using the helper functions now ...
inDir = "/Users/tmo22/Documents/JapaneseProject/RawData/"
projectDirs = ['sspData/', 'borehole.farm/']
yearDirs = ['2005/', '2006/', '2007/']
outDir = "/Users/tmo22/Documents/JapaneseProject/ProcData/"


# Select particular project-Year-Dir - (iterate in the future) - and parse to method to return list of files
projectDir = inDir + projectDirs[0]
for yearDir in yearDirs[1:3]:
#for yearDir in yearDirs:

    # stationFiles = iterateProjectData(projectDir, yearDir)
    [stations, listFilebyStation, orientationCde] = iterateProjectData(projectDir, yearDir)


    #for iterStation in range(8,11,1):
    for iterStation in range( len(stations) ):
        # Use shell command to recover all vertical records
        DataDir = projectDir + yearDir + stations[iterStation] + '/'
        DataFiles = listFilebyStation[iterStation]

        numRecs = len(DataFiles)

        listGoodTraces = []
        cntGoodTraces = 0

        #sys.exit("Demo for station iteration ..")
        stationAzim = orientationCde[stations[iterStation]]

        #for iterDataFile in range(3):
        for iterDataFile in range(len(DataFiles)):

            # Pick out file name ...
            DataFile = DataFiles[iterDataFile]
            fileSplit = DataFile.split('/'); indxLast = len(fileSplit); fileName =  fileSplit[indxLast-1];
            fileHead = fileName[0:len(fileName)-1]

            # ....................................0. Display SCAN status
            print "Dir: ", yearDir
            print "Station: ", stations[iterStation]
            print "H1 Orientation Azim ", stationAzim
            print "Number of records: ", numRecs;
            print "Reading ", DataDir + fileHead, "File ", iterDataFile, " of ", len(DataFiles)
            print "No of QC tagged records: ", cntGoodTraces, "List:  ", listGoodTraces

            #  ................................... 1. READ!
            threeCompStrm = read3Channels(DataDir, fileHead, "SAC")
            #threeCompStrm[0].plot(); threeCompStrm[1].plot(); threeCompStrm[2].plot();

            # Trace slicing parameters.........  2. prePROCESS!
            noiseWindow = 100; signalWindow = 600
            procStrm = preProcessStream(threeCompStrm,noiseWindow, signalWindow)
            #procStrm[0].plot(type="relative", number_of_ticks=4);
            #procStrm[1].plot(type="relative", number_of_ticks=4);
            #procStrm[2].plot(type="relative", number_of_ticks=4);

             # ............................ 3.ROTATE!
            rotateStream2ZRT(procStrm,stationAzim)
            #procStrm[0].plot(type="relative", number_of_ticks=4);  procStrm[1].plot(type="relative", number_of_ticks=4);
            #procStrm[2].plot(type="relative", number_of_ticks=4);
            #print readHeaders(procStrm[0])


            # ............................. 4. doQC and TAGPHASE ..
            isGood = runBandCheckQC(procStrm, noiseWindow, signalWindow)

            if (isGood):
                listGoodTraces.append(iterDataFile)
                cntGoodTraces += 1

            procStrm[0].plot(type="relative", number_of_ticks=4)

            #time.sleep(1)
            clear_output()


            # ............................. 5. saveSTREAM to outputDIR ..
            outDirSave = outDir + projectDirs[0] + yearDir + stations[iterStation] + '/'
            saveStream(procStrm, outDirSave, fileHead)





Saving trace to:  /Users/tmo22/Documents/JapaneseProject/ProcData/sspData/2006/t05/t05.061129.153844..[ZRT]
Dir exists: 
Dir:  2006/
Station:  t05
H1 Orientation Azim  100.041
Number of records:  597
Reading  /Users/tmo22/Documents/JapaneseProject/RawData/sspData/2006/t05/t05.061129.230815. File  63  of  597
No of QC tagged records:  8 List:   [1, 3, 6, 8, 15, 42, 43, 61]
Names:  t05.061129.230815.Z t05.061129.230815.H1 t05.061129.230815.H2
H1 azim: 

 100.041 H2 azim:  190.041
Signal to Noise: 

 1800.31946106 4029.08689183 0.446830636665
Signal to Noise: 

 18114.7004697 16272.7763689 1.11319052502
Signal to Noise: 

 31121.4100065 34473.4983411 0.902763325572


In [ ]:
21 # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# Author: Tolulope Olugboji
# Date:  October 10, 2013
#
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#
# Objective:  Read SEED files from Kawakatsu BoreHole Data
#             Scan a huge record database and parse event times, Conduct Quality Check,
#             and visualize seismograms semi-autonomously
#
#
# Code scans through data and displays each band pass window for the vertical seismogram
# Modules include dateTime, read_Records, signal_filtering, envelopes, output_Clear, tauP_routines
#
#

from obspy import UTCDateTime
from obspy.xseed import Parser
from obspy import read
from obspy.core.util import locations2degrees
from obspy.core.util.geodetics import gps2DistAzimuth
import obspy.signal
import time, sys, os
from IPython.display import display, clear_output
from obspy.taup.taup import getTravelTimes

def updateSacHeaders(trace, sacDics):

    #SAC Dics:  {'elevation': -6036.7,
    #            'evlon': 175.583,
    #            'evlat': -18.343,
    #            'longitude': 159.9632,
    #            'evDep': 251.0,
    #            'local_depth': 0.0,
    #            'latitude': 41.0797,
    #            'evMag': 5.6}

    print "SAC DICS In the trace updater...", sacDics
    trace.stats.sac['evla'] = sacDics['evlat']
    trace.stats.sac['evlo'] = sacDics['evlon']
    trace.stats.sac['evdp'] = sacDics['evDep']

    trace.stats.sac['stla'] = sacDics['latitude']
    trace.stats.sac['stlo'] = sacDics['longitude']
    trace.stats.sac['stdp'] = sacDics['elevation']

    trace.stats.sac['gcarc'] = locations2degrees( sacDics['latitude'], sacDics['longitude'],
                                                 sacDics['evlat'], sacDics['evlon'] )

    dist, az, baz = gps2DistAzimuth(sacDics['evlat'], sacDics['evlon'], sacDics['latitude'],
                                   sacDics['longitude'] )

    trace.stats.sac['az'] = az
    trace.stats.sac['baz'] = baz

    trace.stats.sac['mag'] = sacDics['evMag']


    # Update Component Azimuth, Useful for external rotation routine ...
    if ( trace.stats['channel'] == 'BHN' ):
        trace.stats.sac['cmpaz'] = 0.0
    elif ( trace.stats['channel'] == 'BHE' ):
        trace.stats.sac['cmpaz'] = 90.0
    elif ( trace.stats['channel'] == 'BHZ' ):
        trace.stats.sac['cmpaz'] = 0.0




def saveStream(seedStrm, outDir, fileMantissa, evDics, fileFormat):
    # Code Snippet to Take Processed Stream and Save to Hard-Disk"
    "Save Stream to Directory, Default Output is SAC file ..."
    print "Saving trace to: ", outDir+fileMantissa+".[ZRT]"

    # Station routing logic - Save - LoadSAC - UpdateHeaders - Save
    for trace in seedStrm:
        if (trace.stats['station'] == 'WP1'):
            fDir = outDir+'WP1/'+fileMantissa+trace.stats['channel']
            print "WP1 Folder ..", fDir

            # Stitch dictionaries together ...
            stDics = {'latitude': 19.2975, 'local_depth': 0.0, 'elevation': -6282.2, 'longitude': 135.0992}
            sacDics = dict( evDics.items()+stDics.items() )


            # Route Trace to appropriate Directory [Create if necessary]
            if (os.path.isdir(outDir + 'WP1/') ):
                print "Dir exists: "

                # .................................     Save
                trace.write(fDir, format=fileFormat)

                # .................................     LOAD SAC
                noHeaders = read(fDir)

                # .................................     Update Headers...
                updateSacHeaders(noHeaders[0], sacDics)

                # .................................     Save
                noHeaders[0].write(fDir, format=fileFormat)
            else:
                print "Make Directory, then save.."
                os.makedirs(outDir + 'WP1/')
                # .................................     Save
                trace.write(fDir, format=fileFormat)

                # .................................     LOAD SAC
                noHeaders = read(fDir)

                # .................................     Update Headers...
                updateSacHeaders(noHeaders[0], sacDics)
                #print noHeaders[0].stats['sac']

                # .................................     Save
                noHeaders[0].write(fDir, format=fileFormat)


        elif (trace.stats['station'] == 'WP2'):
            fDir = outDir+'WP2/'+fileMantissa+trace.stats['channel']
            print "WP2 Folder ..", fDir

            # Stitch dictionaries together ...
            stDics = {'latitude': 41.0797, 'local_depth': 0.0, 'elevation': -6036.7, 'longitude': 159.9632}
            sacDics = dict( evDics.items()+stDics.items() )


            # Route Trace to appropriate Directory [Create if necessary]
            if (os.path.isdir(outDir + 'WP2/') ):
                print "Dir exists: "
                # .................................     Save
                trace.write(fDir, format=fileFormat)

                # .................................     LOAD SAC
                noHeaders = read(fDir)

                # .................................     Update Headers...
                updateSacHeaders(noHeaders[0], sacDics)
                #print noHeaders[0].stats

                # .................................     Save
                noHeaders[0].write(fDir, format=fileFormat)

            else:
                print "Make Directory, then save.."
                os.makedirs(outDir + 'WP2/')
                # .................................     Save
                trace.write(fDir, format=fileFormat)

                # .................................     LOAD SAC
                noHeaders = read(fDir)

                # .................................     Update Headers...
                updateSacHeaders(noHeaders[0], sacDics)
                #print noHeaders[0].stats

                # .................................     Save
                noHeaders[0].write(fDir, format=fileFormat)

        #print "SAC Dics: ", sacDics

def getEventCordinates(line):


    if ( not(line) ):
        # Code for earthquakes without event records ...
        sacDic = {'evlat': 0.0, 'evlon': 0.0, 'evMag': 0.0, 'evDep': 0.0}
    else:
        cols = line.split()

        evlat = float(cols[1])
        evlon = float(cols[3])

        if (cols[2] == 'S'):
            evlat = evlat * -1.0
        elif(cols[4] == 'W'):
            evlon = evlon * -1.0

        evMag = float(cols[6])
        evDep = float(cols[5])

        print cols
        print "Event Lat", evlat, "Evnt Lon", evlon, "Mag", evMag, "Depth", evDep

        sacDic = {'evlat': evlat, 'evlon': evlon, 'evMag': evMag, 'evDep': evDep}

    return sacDic




def getSACEventDic(timeStart):

    dataYr = UTCDateTime(timeStart).year
    lookUpDateStr = UTCDateTime(timeStart).strftime("%Y%m%d%H")
    print "Year: ", dataYr

    foundDate = False;
    evntInfo = getEventCordinates("") # Initialize event info to zero for no match ..



    # Look up the eventfiles using year
    inEventDir = "/Users/tmo22/Documents/JapaneseProject/RawData/borehole.farm/eventlist/"
    if(dataYr == 2000):
        # read 2000list
        fName = inEventDir + "2000all.list"
        eventFile = open(fName, 'r');
        eventTable = eventFile.readlines()
        eventFile.close()

        for line in eventTable:

            yr = int( line[0:4] )
            month = int( line[4:6] )
            day = int( line[6:8] )
            hr = int( line[8:10] )
            mins = int( line[10:12] )

            eventDate = UTCDateTime(yr, month, day, hr, mins)
            #print yr, month, day, hr, mins

            fileDateStr =  UTCDateTime(eventDate).strftime("%Y%m%d%H");
            foundDate = UTCDateTime(fileDateStr) == UTCDateTime(lookUpDateStr)
            #print "Value on File YrMnthDayHrMin: ", fileDateStr, "Found? ", cmprDate
            #print "Look up Date Str: ", fileDateStr, "String initialize: ", line[0:12]

            if (foundDate):
                print "Look up Date [Data]: ", UTCDateTime(timeStart).year, UTCDateTime(timeStart).month, UTCDateTime(timeStart).day, UTCDateTime(timeStart).hour, UTCDateTime(timeStart).minute
                print "Look up Date [Event]: ", yr, month, day, hr, mins
                print "Event Info: ", line

                ## Read up Event Long & Latitude
                evntInfo = getEventCordinates(line)
                break

    elif(dataYr == 2001):
        # read 2001list
        fName = inEventDir + "2001all.list"
        eventFile = open(fName, 'r');
        eventTable = eventFile.readlines()
        eventFile.close()

        for line in eventTable:

            yr = int( line[0:4] )
            month = int( line[4:6] )
            day = int( line[6:8] )
            hr = int( line[8:10] )
            mins = int( line[10:12] )

            eventDate = UTCDateTime(yr, month, day, hr, mins)
            #print "Look up Date [Event]: ", yr, month, day, hr, mins

            fileDateStr =  UTCDateTime(eventDate).strftime("%Y%m%d%H");
            #print "Event Date Str: ", fileDateStr, "Data Date Str: ", lookUpDateStr

            foundDate = ( fileDateStr == lookUpDateStr )
            #print "Value on File YrMnthDayHrMin: ", fileDateStr, "Found? ", cmprDate


            if (foundDate):
                print "Look up Date [Data]: ", UTCDateTime(timeStart).year, UTCDateTime(timeStart).month, UTCDateTime(timeStart).day, UTCDateTime(timeStart).hour, UTCDateTime(timeStart).minute
                print "Look up Date [Event]: ", yr, month, day, hr, mins
                print "Event Info: ", line

                ## Read up Event Long & Latitude
                evntInfo = getEventCordinates(line)
                break

    elif(dataYr == 2002):
        # read 2002list
        fName = inEventDir + "2002all.list"
        eventFile = open(fName, 'r');
        eventTable = eventFile.readlines()
        eventFile.close()

        for line in eventTable:

            yr = int( line[0:4] )
            month = int( line[4:6] )
            day = int( line[6:8] )
            hr = int( line[8:10] )
            mins = int( line[10:12] )

            eventDate = UTCDateTime(yr, month, day, hr, mins)

            fileDateStr =  UTCDateTime(eventDate).strftime("%Y%m%d%H");
            #foundDate = UTCDateTime(fileDateStr) == UTCDateTime(lookUpDateStr)
            foundDate = ( fileDateStr == lookUpDateStr )
            #print "Value on File YrMnthDayHrMin: ", fileDateStr, "Found? ", cmprDate
            #print "Look up Date Str: ", fileDateStr, "String initialize: ", line[0:12]

            if (foundDate):
                print "Look up Date [Data]: ", UTCDateTime(timeStart).year, UTCDateTime(timeStart).month, UTCDateTime(timeStart).day, UTCDateTime(timeStart).hour, UTCDateTime(timeStart).minute
                print "Look up Date [Event]: ", yr, month, day, hr, mins
                print "Event Info: ", line

                ## Read up Event Long & Latitude
                evntInfo = getEventCordinates(line)
                break

    elif(dataYr == 2003):
        # read 2003list

        fName = inEventDir + '2003all.list'
        eventFile = open(fName, 'r');
        eventTable = eventFile.readlines()
        eventFile.close()

        for line in eventTable:

            yr = int( line[0:4] )
            month = int( line[4:6] )
            day = int( line[6:8] )
            hr = int( line[8:10] )
            mins = int( line[10:12] )

            eventDate = UTCDateTime(yr, month, day, hr, mins)

            fileDateStr =  UTCDateTime(eventDate).strftime("%Y%m%d%H");
            #foundDate = UTCDateTime(fileDateStr) == UTCDateTime(lookUpDateStr)
            foundDate = ( fileDateStr == lookUpDateStr )
            #print "Value on File YrMnthDayHrMin: ", fileDateStr, "Found? ", cmprDate
            #print "Look up Date Str: ", fileDateStr, "String initialize: ", line[0:12]

            if (foundDate):
                print "Look up Date [Data]: ", UTCDateTime(timeStart).year, UTCDateTime(timeStart).month, UTCDateTime(timeStart).day, UTCDateTime(timeStart).hour, UTCDateTime(timeStart).minute
                print "Look up Date [Event]: ", yr, month, day, hr, mins
                print "Event Info: ", line

                ## Read up Event Long & Latitude
                evntInfo = getEventCordinates(line)
                break

    elif(dataYr == 2004):
        # read 2004list
        fName = inEventDir + "2004all.list"
        eventFile = open(fName, 'r');
        eventTable = eventFile.readlines()
        eventFile.close()

        for line in eventTable:

            yr = int( line[0:4] )
            month = int( line[4:6] )
            day = int( line[6:8] )
            hr = int( line[8:10] )
            mins = int( line[10:12] )

            eventDate = UTCDateTime(yr, month, day, hr, mins)

            fileDateStr =  UTCDateTime(eventDate).strftime("%Y%m%d%H");
            #foundDate = UTCDateTime(fileDateStr) == UTCDateTime(lookUpDateStr)
            foundDate = ( fileDateStr == lookUpDateStr )
            #print "Value on File YrMnthDayHrMin: ", fileDateStr, "Found? ", cmprDate
            #print "Look up Date Str: ", fileDateStr, "String initialize: ", line[0:12]

            if (foundDate):
                print "Look up Date [Data]: ", UTCDateTime(timeStart).year, UTCDateTime(timeStart).month, UTCDateTime(timeStart).day, UTCDateTime(timeStart).hour, UTCDateTime(timeStart).minute
                print "Look up Date [Event]: ", yr, month, day, hr, mins
                print "Event Info: ", line

                ## Read up Event Long & Latitude
                evntInfo = getEventCordinates(line)
                break

    if ( not (foundDate) ):
        print "LookUp Failed"
    return dataYr, evntInfo


########################### WorkFlow commences here .......................
# ..........................................................................

rootDir = "/Users/tmo22/Documents/JapaneseProject/RawData/"
projectDir = ['sspData/', 'borehole.farm/']


# Directory to demo processing: output store for procData
outDir = "/Users/tmo22/Documents/JapaneseProject/ProcData/boreHole/"

outDir = rootDir +  projectDir[1] + 'farmSAC/'# Out SAC First
DataDir = rootDir + projectDir[1] + 'farm'

# Use shell command to recover all files ...
DataFiles = !ls $DataDir
numRecs = len(DataFiles)
numRecs = 1

print DataDir
print "Number of records: ", numRecs;

cntTraces = 1
cntNoEvnts = 0
listBadEvnts = []

for DataFile in DataFiles[365:366]:
#for DataFile in DataFiles:


    clear_output(); stationDic = []

    # Reed SEED Automatically. Also Update the Station Dictionary (MetaData) ..
    print "Data Files: ", DataFile, cntTraces, " of ", len(DataFiles)

    seedStrm = read(DataDir+'/'+DataFile)
    seedMeta = Parser(DataDir+'/'+DataFile)
    #print seedMeta.getCoordinates('PS.WP1..BHE')

    # two station read coordinates !!!!! FIX  BUG HERE>>>
    #if ( len(seedMeta.getInventory()['stations']) == 2 ):
    #    stationDic.append( seedMeta.getCoordinates('PS.WP1..BHE') )
    #    stationDic.append( seedMeta.getCoordinates('PS.WP2..BHE') )
    #else:
    #    stationDic.append( seedMeta.getCoordinates('BHE') )


    #print(seedStrm)


    # Parse File Mantissa
    fileSplit = DataFile.split('.'); indxLast = len(fileSplit);
    fileHead =  fileSplit[0] + '.';
    print fileHead

    # Save Output in SAC

    timeLkUp = seedStrm[0].stats.starttime
    [yr, evntDic] = getSACEventDic(timeLkUp)
    #sacDics = dict( evntDic.items() + stationDic.items() )




    # Save stream !!! Rewrite function signature to seperate event dictionaries, and station dictionaries
    saveStream(seedStrm, outDir, fileHead, evntDic, "SAC")

    #seedStrm.plot()
    #time.sleep(3)

    cntTraces += 1

    if (evntDic['evlat'] == 0.0):
        cntNoEvnts += 1
        listBadEvnts.append(fileHead)

    print "Records with No Events: ", cntNoEvnts, listBadEvnts


Data Files:  200206110422.seed 1  of  924
6 Trace(s) in Stream:
PS.WP1..BHE | 2002-06-11T04:22:23.800000Z - 2002-06-11T06:23:53.550000Z | 20.0 Hz, 145796 samples
PS.WP1..BHN | 2002-06-11T04:21:05.150000Z - 2002-06-11T06:22:48.950000Z | 20.0 Hz, 146077 samples
PS.WP1..BHZ | 2002-06-11T04:21:53.050000Z - 2002-06-11T06:23:33.350000Z | 20.0 Hz, 146007 samples
PS.WP2..BHE | 2002-06-11T04:21:21.950000Z - 2002-06-11T06:22:54.650000Z | 20.0 Hz, 145855 samples
PS.WP2..BHN | 2002-06-11T04:21:08.950000Z - 2002-06-11T06:22:42.950000Z | 20.0 Hz, 145881 samples
PS.WP2..BHZ | 2002-06-11T04:22:13.600000Z - 2002-06-11T06:23:48.800000Z | 20.0 Hz, 145905 samples


200206110422.
Year:  2002
Look up Date [Data]:  2002 6 11 4 22
Look up Date [Event]:  2002 6 11 4 22
Event Info:  20020611042238.2  58.637 N  155.140 W  132 5.5 ALASKA PENINSULA, UNITED STATES                      

['20020611042238.2', '58.637', 'N', '155.140', 'W', '132', '5.5', 'ALASKA', 'PENINSULA,', 'UNITED', 'STATES']
Event Lat 58.637 Evnt Lon -155.14 Mag 5.5 Depth 132.0
Saving trace to:  /Users/tmo22/Documents/JapaneseProject/RawData/borehole.farm/farmSAC/200206110422..[ZRT]
WP1 Folder .. /Users/tmo22/Documents/JapaneseProject/RawData/borehole.farm/farmSAC/WP1/200206110422.BHE
Dir exists: 
SAC DICS In the trace updater... {'elevation': -6282.2, 'evlon': -155.14, 'evlat': 58.637, 'longitude': 135.0992, 'evDep': 132.0, 'local_depth': 0.0, 'latitude': 19.2975, 'evMag': 5.5}
WP1 Folder .. /Users/tmo22/Documents/JapaneseProject/RawData/borehole.farm/farmSAC/WP1/200206110422.BHN
Dir exists: 
SAC DICS In the trace updater... {'elevation': -6282.2, 'evlon': -155.14, 'evlat': 58.637, 'l

 {'elevation': -6036.7, 'evlon': -155.14, 'evlat': 58.637, 'longitude': 159.9632, 'evDep': 132.0, 'local_depth': 0.0, 'latitude': 41.0797, 'evMag': 5.5}
Records with No Events:  0 []


In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# Author: Tolulope Olugboji
# Date:  October 10, 2013
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#
# Objective:   Read SAC Files [BOREHOLE DATA ONLY] from Kawakatsu Japanese project...
#              Scan a huge record database and parse event times, Conduct Quality Check,
#             and visualize seismograms semi-autonomously
#
#
# Code scans through data and displays each band pass window for the vertical seismogram
# Modules include dateTime, read_Records, signal_filtering, envelopes, output_Clear, tauP_routines
#
#

from obspy import Stream
from obspy import UTCDateTime
from obspy import read
import obspy.signal
from obspy.signal import rotate_new
import time, sys, os
from IPython.display import display, clear_output
from obspy.taup.taup import getTravelTimes

def smooth(x,window_len=11,window='hanning'):
    """ http://wiki.scipy.org/Cookbook/SignalSmoothsmooth the data using a window with requested size.

    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.

    input:
        x: the input signal
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal

    example:

    t=linspace(-2,2,0.1)
    x=sin(t)+randn(len(t))*0.1
    y=smooth(x)

    see also:

    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter

    TODO: the window parameter could be the window itself if an array instead of a string
    NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
    """

    if x.ndim != 1:
        raise ValueError, "smooth only accepts 1 dimension arrays."

    if x.size < window_len:
        raise ValueError, "Input vector needs to be bigger than window size."


    if window_len<3:
        return x


    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError, "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'"


    s=numpy.r_[x[window_len-1:0:-1],x,x[-1:-window_len:-1]]
    #print(len(s))
    if window == 'flat': #moving average
        w=numpy.ones(window_len,'d')
    else:
        w=eval('numpy.'+window+'(window_len)')

    y=numpy.convolve(w/w.sum(),s,mode='valid')

    edge = int( (window_len)/2 )
    #print "edge: ", edge
    return y[edge:len(y)-edge]


def findTriggerTrace(trace, S2NThreshold):
    t0 = 0.0
    delta = float(trace.stats.delta)
    nPoints = int(trace.stats.npts)
    nVec = np.array( range(nPoints) )
    tt = t0 + nVec*delta
    traceData = trace.data
    trEnv = obspy.signal.filter.envelope(traceData)


    # Compute STA/LTA
    tNoise = 20 # secs
    tSig = 5  #secs

    nNoise = tNoise/delta
    nSig = tSig/delta
    nWin = nNoise+nSig

    trigger = []

    for iPoint in nVec[0:nWin]:
        trigger.append(0.0)

    for iPoint in nVec[nWin:nPoints]:
        LTA = np.average( trEnv[iPoint-nNoise:iPoint] )
        STA = np.average( trEnv[iPoint: iPoint+nSig] )

        trigger.append(STA/LTA)

    smoothTrig = smooth(np.array(trigger), nSig+1)
    nTrig = len(trigger)
    findTrig = 0

    # Define search betweeen 100 secs and 240 secs?
    nStart = int(100/delta)
    nStop = int(240/delta)
    for iTrig in range(nStart,nStop,1):
        if ( smoothTrig[iTrig] > S2NThreshold):
            findTrig = iTrig
            break

    # Return timing of phase, for use by BandCheck, and traceTagQC...
    return tt[findTrig], smoothTrig

def traceTagTimeSlow(threeCompStrm, travelTimeDic, pTime):
    "tag the header t0 with first arrival phase name"

    # Write Phase Name, Phase Slowness and Phase ?
    for iCmp in range(3):
        #print "trace: ", iCmp, " of ", len(threeCompStrm)
        threeCompStrm[iCmp].stats.sac['t0'] = pTime
        threeCompStrm[iCmp].stats.sac['user0'] =  travelTimeDic['dT/dD']
        threeCompStrm[iCmp].stats.sac['kt0'] =  travelTimeDic['phase_name']


def traceTagQC(threeCompStrm, pTime):
    "tag the header t1, to indicate QC pass"

    # Put QC tag in here ...
    for iCmp in range(3):
        threeCompStrm[iCmp].stats.sac['t1'] = pTime

    # Write a phase picking routine hear new ...


def read3Channels(DataDir, fileMantissa, Fileformat):
    #Code snippet to take file name, and return 3component Stream.
    "Code snippet to take file name, and return 3component Stream."
    vertFile = fileMantissa+"BHZ";
    radFile = fileMantissa+"BHN";
    transFile = fileMantissa+"BHE"
    #print "Names: ", vertFile, radFile, transFile;
    # read each record triplet, and display ...
    threeCompStrm = read(DataDir + '/' + vertFile);
    threeCompStrm += read(DataDir + '/' + radFile);
    threeCompStrm += read(DataDir + '/' + transFile);
    return threeCompStrm

def preProcessStream(threeCompStrm, noiseWindowSec, signalWindowSec):
    "Slice Trace, Demean, Detrend, and HighPass?"
    # noiseWindowSec : time before arrival P time
    # signalWindowSec : time after arrival P time

    #!!!! ................................. Load Depth, Distance, Then Compute Travel Time
    evntDistDeg = threeCompStrm[0].stats.sac['gcarc'];
    evntDep = threeCompStrm[0].stats.sac['evdp'];
    strtTime = threeCompStrm[0].stats['starttime'];
    tt = getTravelTimes(delta=evntDistDeg, depth = evntDep, model='ak135')
    pTime = tt[0]['time']

    biasLeft = noiseWindowSec; biasRight = signalWindowSec
    #print "Windows ...", biasLeft, biasRight, "Time ", strtTime

    #1. Slice And Tag ...
    #threeTrace = []
    #for iCmp in range(3):
    procStream =  threeCompStrm.slice(strtTime+pTime - biasLeft, strtTime+pTime+biasRight,
                                            keep_empty_traces=False)

    if (len(procStream) == 3):
        #procStream = Stream(traces=threeTrace)
        #print procStream
        traceTagTimeSlow(procStream, tt[0], biasLeft)

        # Demean, Detrend, and, HighPass? Yes with 50s [1/50 = 0.02 Hz]  *******
        lowFreq = 0.02
        procStream.detrend(type="demean"); procStream.detrend(type="simple")
        procStream.filter("highpass", freq=lowFreq)
        #print procStream
        return procStream
    else:
        #print "Slice failed!"
        return procStream






def readHeaders(trace):
    "Read relevant header information ..."
    return {'gcarc': trace.stats.sac['gcarc'], 'evdp':trace.stats.sac['evdp'],
            'starttime':trace.stats['starttime'], 'o': trace.stats['sac']['o'], 'delta': trace.stats['delta'],
            'baz':trace.stats.sac['baz']}

def rotateStream2ZRT(threeCompStrm, H1Azim):
    "Load 3 component stream and rotate from ZNE into ZRT."

    #rotate_new from krischer / obspy might be useful ..
    baz = threeCompStrm[1].stats.sac['baz']


    dipZcomp = -90; dipH1comp = 0.0; dipH2comp = 0.0
    azimZcomp = 0; azimH1comp = H1Azim; azimH2comp = (H1Azim + 90.0) % 360
    #print "H1 azim: ", azimH1comp, "H2 azim: ", azimH2comp
    zData = threeCompStrm[0].data ; H1Data = threeCompStrm[1].data ; H2Data = threeCompStrm[2].data;

    [Z, N, E] = rotate_new.rotate2ZNE(zData, azimZcomp, dipZcomp, H1Data, azimH1comp, dipH1comp, \
                                      H2Data,azimH2comp, dipH2comp)

    #N = threeCompStrm[1].data ; E = threeCompStrm[2].data;


    [R, T] = rotate_new.rotate_NE_RT(N, E, baz)

    threeCompStrm[0].data = Z
    threeCompStrm[1].data = R
    threeCompStrm[2].data = T



def saveStream(threeCompStrm, outDir, fileMantissa):
    # Code Snippet to Take Processed Stream and Save to Hard-Disk"
    "Save Stream to Directory, Default Output is SAC file ..."
    #print "Saving trace to: ", outDir+fileMantissa+".[ZRT]"

    vertFile = fileMantissa+"Z";
    radFile = fileMantissa+"R";
    transFile = fileMantissa+"T"

    if (os.path.isdir(outDir) ):
        #print "Dir exists: "
        threeCompStrm[0].write(outDir + vertFile, format="SAC")
        threeCompStrm[1].write(outDir + radFile, format="SAC")
        threeCompStrm[2].write(outDir + transFile, format="SAC")
    else:
        #print "Make Directory, then save.."
        os.makedirs(outDir)
        threeCompStrm[0].write(outDir + vertFile, format="SAC")
        threeCompStrm[1].write(outDir + radFile, format="SAC")
        threeCompStrm[2].write(outDir + transFile, format="SAC")


def runBandCheckQC(threeCompStrm, noiseWin, signalWin):
    from copy import deepcopy
    # run signal to noise in each window and tag phas appropriately
    #################### FREQ Windows
    cornerFreq = 0.07
    lowFreqA = 0.07
    highFreqA = 0.25
    lowFreqB = 0.25
    highFreqB = 2
    #################### End FREQ Windows

    # Make copy of stream ...
    lowFrqStrm = deepcopy(threeCompStrm)
    midFrqStrm = deepcopy(threeCompStrm)
    highFrqStrm = deepcopy(threeCompStrm)

    # Band Pass these Copies
    #lowFrqStrm.filter("lowpass", freq=cornerFreq);
    lowFrqStrm.filter("bandpass", freqmin=0.01, freqmax=0.07, corners=2);
    midFrqStrm.filter("bandpass", freqmin=lowFreqA, freqmax=highFreqA);
    highFrqStrm.filter("bandpass", freqmin=lowFreqB, freqmax=highFreqB);

    zTraceBands = []
    zEnvlpBands = []
    zComps = []
    allSigNoise = []

    # Default trigger Time to predicted time for P arrival from TauP
    trigTimeBands = [100, 100, 100]
    colors = ['b', 'g', 'k']
    pickFuncBands = []

    # Pick Out vertical Components of Bands and append to list for signal analysis
    zTraceBands.append(lowFrqStrm[0])
    zEnvlpBands.append( obspy.signal.filter.envelope(lowFrqStrm[0].data) )
    zComps.append( lowFrqStrm[0].data )

    zTraceBands.append( midFrqStrm[0] )
    zEnvlpBands.append( obspy.signal.filter.envelope(midFrqStrm[0].data) )
    zComps.append( midFrqStrm[0].data )

    zTraceBands.append( highFrqStrm[0] )
    zEnvlpBands.append( obspy.signal.filter.envelope(highFrqStrm[0].data) )
    zComps.append( highFrqStrm[0].data )


    plt.figure(figsize=(10,4))

    # Iterate through bands and do signal to noise computation ...
    envMax = 0
    envMin = 0
    for iterBands in range( len(zEnvlpBands) ):

        plt.subplot(4,1,iterBands+1)

        zTraceBand = zTraceBands[iterBands]
        zEnvlpBand = zEnvlpBands[iterBands]
        zComp = zComps[iterBands]

        t = np.linspace(0, noiseWin+signalWin, len(zEnvlpBand) )
        noise =  zEnvlpBand[t<100];  signal =  zEnvlpBand[t >100];

        sig2noise = np.average(np.absolute(signal)) / np.average(np.absolute(noise));
        allSigNoise.append( sig2noise )
        print "Signal to Noise: ", np.average(np.absolute(signal)), np.average(np.absolute(noise)), sig2noise

        plt.plot(t, zComp, colors[iterBands])
        plt.plot(t, zEnvlpBand, 'm')

        envMax = np.amax( zEnvlpBands )
        envMin = np.amin( zComps )



        # If good ... Calculate trigger and plot too ..
        maxS2N = 1.6
        trigThreshold = 2.0

        # There should be 3 triggers for each frequency window ...
        tTrig, pickFunc = findTriggerTrace(zTraceBand, trigThreshold)
        trigTimeBands[iterBands] = tTrig
        pickFuncBands.append(pickFunc)

        if( sig2noise > maxS2N):
            plt.plot(t[0:len(noise)], noise, 'r')



    sig2noiseMax = np.amax(allSigNoise)
    bandWithMax = np.argmax(allSigNoise)
    print "Maximum Signal to Noise across 3 Frequency bands: ", sig2noiseMax


    if( sig2noiseMax > maxS2N):
        #traceTagQC(threeCompStrm, noiseWin)  ## OLD TAG WITH 100sec. Predicted Arrival
        traceTagQC(threeCompStrm, trigTimeBands[bandWithMax])
        tagStatus = True

        # Print predicted timing for trigger ...
        plt.vlines(trigTimeBands[bandWithMax], envMin, envMax)

    else:
        tagStatus = False

    plt.subplot(4,1,4)
    plt.plot(t, pickFuncBands[0], colors[0] )
    plt.plot(t, pickFuncBands[1], colors[1] )
    plt.plot(t,  pickFuncBands[2], colors[2])
    plt.hlines(trigThreshold, 0,299)
    plt.show()
    return tagStatus;


########################### END OF FUNCTION DEFINITIONS #########################################################
#!!!!!!!!!!!!!!!!!!!!!!!!!!! BEGIN WORKFLOW HERE ...... !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
##++--- ---+ !!!!!!! Work on Work flow using the helper functions now ...
inDir = "/Users/tmo22/Documents/JapaneseProject/RawData/borehole.farm/farmSAC/"
stationDirs = ['WP1/', 'WP2/']

outDir = "/Users/tmo22/Documents/JapaneseProject/ProcData/borehole/"


# Select particular station - and parse to method to return list of files

stationAzim = {'WP1/':315, 'WP2/':156}

for stationDir in stationDirs[0:1]:
#for stationDir in stationDirs:

    for iterStation in range(0,1,1):
    #for iterStation in range( len(stations) ):
        # Use shell command to recover all vertical records
        DataDir = inDir + stationDirs[iterStation]
        Hor1Azim = stationAzim[ stationDirs[iterStation] ]
        DataFiles = !ls $DataDir/*Z

        numRecs = len(DataFiles)

        listGoodTraces = []
        cntGoodTraces = 0

        cntFail = 0


        #for iterDataFile in range(0,20,1):
        for iterDataFile in range(len(DataFiles)):

            # Pick out file name ...
            DataFile = DataFiles[iterDataFile]
            fileSplit = DataFile.split('/'); indxLast = len(fileSplit); fileName =  fileSplit[indxLast-1];
            fileHead = fileName[0:len(fileName)-3]

            # ....................................0. Display SCAN status
            print "Dir: ", inDir
            print "Station: ", stationDirs[iterStation]
            print "H1 Orientation Azim ", Hor1Azim
            print "Number of records: ", numRecs;
            print "Reading ", DataDir + fileHead, "File ", iterDataFile, " of ", len(DataFiles)
            print "No of QC tagged records: ", cntGoodTraces, "List:  ", listGoodTraces
            print "No of records fail slice: ", cntFail

            #  ................................... 1. READ!
            threeCompStrm = read3Channels(DataDir, fileHead, "SAC")
            #threeCompStrm[0].plot(); threeCompStrm[1].plot(); threeCompStrm[2].plot();

            # Trace slicing parameters.........  2. prePROCESS!
            noiseWindow = 100; signalWindow = 600
            procStrm = preProcessStream(threeCompStrm,noiseWindow, signalWindow)

            if (len(procStrm) == 3 and ( len(procStrm[1]) == len(procStrm[2]) ) ):
                #procStrm.plot()
                #procStrm[0].plot(type="relative", number_of_ticks=4);
                #procStrm[1].plot(type="relative", number_of_ticks=4);
                #procStrm[2].plot(type="relative", number_of_ticks=4);

                # ............................ 3.ROTATE! ADD Station Azim Here...
                rotateStream2ZRT(procStrm, Hor1Azim)
                #procStrm[0].plot(type="relative", number_of_ticks=4);  procStrm[1].plot(type="relative", number_of_ticks=4);
                #procStrm[2].plot(type="relative", number_of_ticks=4);
                #print readHeaders(procStrm[0]), #readHeaders(procStrm[1]), readHeaders(procStrm[2])


                # ............................. 4. doQC and TAGPHASE ..
                isGood = runBandCheckQC(procStrm, noiseWindow, signalWindow)

                if (isGood):
                    listGoodTraces.append(iterDataFile)
                    cntGoodTraces += 1

                #procStrm[0].plot(type="relative", number_of_ticks=4)

                #time.sleep(5)
                clear_output()


                # ............................. 5. saveSTREAM to outputDIR ..
                outDirSave = outDir + stationDirs[iterStation]
                saveStream(procStrm, outDirSave, fileHead)
            else:
                cntFail += 1





In [ ]:
# Code to test phase picker using STA/LTA with 5 sec. signal window and (5*4=20 sec.) noise window
# first reconstruct time signal from data headers
#print procStrm[0].stats

# Demo using single good data ...


def smooth(x,window_len=11,window='hanning'):
    """ http://wiki.scipy.org/Cookbook/SignalSmoothsmooth the data using a window with requested size.

    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.

    input:
        x: the input signal
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal

    example:

    t=linspace(-2,2,0.1)
    x=sin(t)+randn(len(t))*0.1
    y=smooth(x)

    see also:

    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter

    TODO: the window parameter could be the window itself if an array instead of a string
    NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
    """

    if x.ndim != 1:
        raise ValueError, "smooth only accepts 1 dimension arrays."

    if x.size < window_len:
        raise ValueError, "Input vector needs to be bigger than window size."


    if window_len<3:
        return x


    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError, "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'"


    s=numpy.r_[x[window_len-1:0:-1],x,x[-1:-window_len:-1]]
    #print(len(s))
    if window == 'flat': #moving average
        w=numpy.ones(window_len,'d')
    else:
        w=eval('numpy.'+window+'(window_len)')

    y=numpy.convolve(w/w.sum(),s,mode='valid')

    edge = int( (window_len)/2 )
    print "edge: ", edge
    return y[edge:len(y)-edge]


procStrm = read("/Users/tmo22/Documents/JapaneseProject/ProcData/boreHole/WP2/200203260345.Z")
t0 = 0.0
delta = float(procStrm[0].stats.delta)
nPoints = int(procStrm[0].stats.npts)
nVec = np.array( range(nPoints) )
tt = t0 + nVec*delta
trace = procStrm[0].data
trEnv = obspy.signal.filter.envelope(trace)

#print type(nVec)
print nVec[nPoints-1:nPoints], tt[nPoints-1:nPoints]
plt.plot(tt, trace, tt, trEnv)
plt.show()


# Compute STA/LTA
tNoise = 20 # secs
tSig = 5  #secs

nNoise = tNoise/delta
nSig = tSig/delta
nWin = nNoise+nSig

trigger = []

for iPoint in nVec[0:nWin]:
    trigger.append(0.0)

for iPoint in nVec[nWin:nPoints]:
    LTA = np.average( trEnv[iPoint-nNoise:iPoint] )
    STA = np.average( trEnv[iPoint: iPoint+nSig] )

    trigger.append(STA/LTA)

nTrig = len(trigger)
trigVal = 2.0
findTrig = 0

for iTrig in range(nTrig):
    if ( trigger[iTrig] > trigVal):
        findTrig = iTrig
        break



convTrig = smooth(np.array(trigger), nSig+1)
print "convLen: ", len(convTrig), "trigLen: ", len(trigger), "sigWIn: ", nSig

#plt.plot(tt[0:nTrig], trigger[0:nTrig], tt, convTrig)
#plt.show()

#print "convLen: ", len(convTrig), "trigLen: ", len(trigger)
#plt.plot(hanTaper)
plt.plot(convTrig)
plt.plot(trigger)
plt.show()

envMax = np.max(trEnv)
envMin = np.min(trace)

plt.plot(tt, trace, tt, trEnv)
plt.vlines(tt[findTrig], envMin, envMax)
plt.show()

[6000] [ 300.]


edge:  50
convLen:  6001 trigLen:  6001 sigWIn:  100.0


In [ ]:
plt.hlines?

In [ ]:
np.amin?

In [ ]:
print seedMeta

Networks:
	PS (Deep-Sea Borehole Stations)
Stations:
	PS.WP2 (Western Pacific borehole station WP2)
Channels:
	PS.WP2..BHE | 100.00 Hz | CMG-1T | 2000-07-11 -  | Lat: 41.1, Lng: 160.0
	PS.WP2..BHN | 100.00 Hz | CMG-1T | 2000-07-11 -  | Lat: 41.1, Lng: 160.0
	PS.WP2..BHZ | 100.00 Hz | CMG-1T | 2000-07-11 -  | Lat: 41.1, Lng: 160.0


In [ ]:
metaA =  seedMeta.getCoordinates("PS.WP2..BHE")
metaB = seedMeta.getCoordinates("PS.WP2..BHE")

print metaA, metaB

metaC = dict(metaA.items() + metaB.items() )

print metaC

{'latitude': 41.0797, 'local_depth': 0.0, 'elevation': -6036.7, 'longitude': 159.9632} {'latitude': 41.0797, 'local_depth': 0.0, 'elevation': -6036.7, 'longitude': 159.9632}
{'latitude': 41.0797, 'local_depth': 0.0, 'elevation': -6036.7, 'longitude': 159.9632}
